In [ ]:
!pip install tqdm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy.io
from tqdm import tqdm
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pickle as pkl

Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
Data_Dir = '/content/drive/My Drive/Data/subj01/sess01'

Data Annotation


In [ ]:
def extract_data(index, class_num):
    inp = []
    Classes = np.array([])
    for i in tqdm(index):
        dir = os.path.join(Data_Dir, 'cond00' + str(i))
        sessionFolders, sessions = list(sorted(os.listdir(dir))), []
        label = np.ones(len(sessionFolders)) * class_num
        Classes = np.append(Classes, label)
        for j in sessionFolders:
            inp.append(scipy.io.loadmat(dir + '/' + j )['F'])
    Data = np.array(inp)
    return Data, Classes

In [ ]:
index_face = np.arange(13,25)
Data_face, label_face = extract_data(index_face, 0)

In [ ]:
index_object = [73, 74, 75, 76, 78, 79, 80, 83, 84, 85, 88, 89]
Data_object, label_object = extract_data(index_object, 1)

In [ ]:
print(Data_face.shape)
print(label_face.shape)
print(Data_object.shape)
print(label_object.shape)

In [ ]:
Data = np.concatenate((Data_face, Data_object))
Label = np.concatenate((label_face, label_object))

Model Fitting

In [ ]:
scaler = StandardScaler()
Final_Acc_Train = []
Final_Acc_Test = []

for j in tqdm(np.arange(0,Data.shape[2])):
    X = Data[:,:,j]
    scaler.fit(X)
    X_norm = scaler.transform(X)
    loo = LeaveOneOut()
    train_acc = []
    test_acc = []
    j = 0
    for train_index, test_index in loo.split(X_norm):

#Leave One Out
        X_train, X_test = X_norm[train_index], X_norm[test_index]    
        y_train, y_test = Label[train_index], Label[test_index]

#Grid Search
        # tuned_parameters = [{'kernel': ['rbf'], 'C': [0.1, 0.5, 1, 10, 100, 1000]},
        #                 {'kernel': ['linear'], 'C': [0.1, 0.5, 1, 10, 100, 1000]}.
        #                 {'kernel': ['Sigmoid'], 'C': [0.1, 0.5, 1, 10, 100, 1000]}]    

        # grid = GridSearchCV(SVC(), tuned_parameters)
        # grid.fit(X_train, y_train);

# GenerateModel (Select first one for grid search)
        # clf = grid.best_estimator_    
        clf = SVC()

#Fit to training
        clf.fit(X_train, y_train)   
        predict_train = clf.predict(X_train)  
        accuracy_train = accuracy_score(y_train, predict_train)    
        train_acc.append(accuracy_train)

#Validate on test
        predict_test = clf.predict(X_test)      
        accuracy_test = accuracy_score(y_test, predict_test)
        test_acc.append(accuracy_test)

#Average accuracy
    Final_Acc_Train.append(sum(train_acc) / len(train_acc))   
    Final_Acc_Test.append(sum(test_acc) / len(test_acc))
    print('accuracy_score on train dataset : ', sum(train_acc) / len(train_acc))
    print('accuracy_score on test dataset : ', sum(test_acc) / len(test_acc))

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Save the accuracy time series

In [ ]:
np.savetxt('/content/drive/My Drive/Train_All_NR.csv',np.array(Final_Acc_Train), header = ' ')
np.savetxt('/content/drive/My Drive/Test_All_NR.csv',np.array(Final_Acc_Test), header = ' ')